coding: utf-8

This jupyter note is a collection of single python files.

You can run them in your own environment.

If you want to run it in a single python file, please enable the comment of 'import...' when copying it to a single python file.

The course homework is in the `/doc/Python 程序设计/课程作业` folder

In [ ]:
import numpy as np
import pandas as pd
import scipy.linalg as la
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.interpolate import interp1d

题 1-1 查找一段 DNA 序列中重复出现的定长序列，要求用类实现

In [ ]:
# solution 1-1.py
class Solution:
    def __init__(self, s: str):
        self.s = s

    def findRepeatDnaSequences(self, n: int = 10) -> list:
        strLen = len(self.s)
        ans = []
        if strLen > n:
            data = {}
            for i in range(strLen - n + 1):
                key = self.s[i : i + n]
                data[key] = 1 if key not in data else data[key] + 1
            ans.append(key for key in data if data[key] > 1)
        return ans


obj = Solution(input())
print(obj.findRepeatDnaSequences())

题 1-2 根据日期输出对应天数

In [ ]:
# solution 1-2.py
(year1, year2), (yyyy, mm, dd) = (
    (
        [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
        [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
    ),
    input(':) 请输入 "year.month.day" 形式的合法日期: ').split("."),
)
print(
    "📅 这一天是 %d 年的第 %d 天" % (int(yyyy), (sum(year2[: int(mm) - 1]) + int(dd)))
) if int(yyyy) % 4 == 0 and int(yyyy) % 100 != 0 or int(yyyy) % 400 == 0 else print(
    "📅 这一天是 %d 年的第 %d 天" % (int(yyyy), sum(year1[: int(mm) - 1]) + int(dd))
)

题 1-3 逆波兰数运算

In [ ]:
# solution 1-3.py
def calcRPN(s: str):
    """
    使用 str.split() 方法分割输入的逆波兰表达式为列表
    使用 str.count() 方法统计运算符的个数
    将列表长度存储为变量避免重复调用 len() 函数
    判断输入的逆波兰表达式是否合法，若不合法则返回错误信息
    考虑到前两位总为数字，count 从列表第三位开始计数并遍历搜索运算符
    若搜索到运算符则使用 eval() 函数将字符串转换为表达式并计算，反之，则搜索下一位
    合并数据树，删除已计算的数据，更新列表长度，回退 count
    直至列表长度为 1，运算完毕，返回结果
    """
    dataAll = s.split()
    dataOperator = (
        dataAll.count("+")
        + dataAll.count("-")
        + dataAll.count("*")
        + dataAll.count("/")
    )
    dataLen = len(dataAll)
    if dataOperator * 2 + 1 != dataLen:
        return "输入的逆波兰表达式有误！"
    count = 2
    while dataLen > 1:
        if dataAll[count] in ("+", "-", "*", "/"):
            dataAll[count] = str(
                eval(dataAll[count - 2] + dataAll[count] + dataAll[count - 1])
            )
            dataAll.pop(count - 1)
            dataAll.pop(count - 2)
            dataLen -= 2
            count -= 1
        else:
            count += 1
    return dataAll[0]


print(calcRPN(input()))

题 2-1 构造一个 10x10 的随机数组，并取出所有 3x3 子矩阵

In [ ]:
# solution 2-1.py
# import numpy as np  # 导入numpy库

# 生成一个10行10列的随机数组
arr = np.empty((10, 10), dtype=np.float64)

# 由行到列的顺序取出所有 3x3 的子数组
for column in range(8):
    for row in range(8):
        print(arr[column : column + 3, row : row + 3])

题 2-2 拟合一个二次函数

In [ ]:
# solution 2-2.py
# import matplotlib.pyplot as plt
# import numpy as np
# import scipy.linalg as la

(a, b, c) = (30, -20, 1)  # 预定义函数系数

# 生成 100 个随机的样品数据，并添加噪音
x = np.linspace(-100, 100, 100)
y = a + b * x + c * x**2 + 1000 * np.random.randn(100)

# 用最小二乘法拟合
A = np.vstack([x**0, x**1, x**2])
sol = la.lstsq(A.T, y)[0]
yFit = sol[0] + sol[1] * x + sol[2] * x**2
print("拟合函数为：y = %f + %f * x + %f * x^2" % (sol[0], sol[1], sol[2]))

# 画图
plt.plot(x, y, "go")
plt.plot(x, yFit, "b")
plt.show()

题 2-3 用三阶样条曲线插值法拟合数据

In [ ]:
# solution 2-3.py
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# from scipy.interpolate import interp1d

# 读取 csv 文件，并转化为 numpy 数组
arrData = np.transpose(np.array(pd.read_csv("./assets/car.csv")))
arrX, arrY = arrData[0], arrData[1]

# 用三阶样条曲线插值法拟合数据
x2y = interp1d(arrX, arrY, "cubic")
arrXNew = np.linspace(0, 1440, 1441)
arrYNew = x2y(arrXNew)

# 梯形积分法求车辆数
print("车辆总数估计为:", int(sum(arrYNew) - arrYNew[0] / 2 - arrYNew[-1] / 2))

# 绘图
plt.plot(arrXNew, arrYNew, color="blue", label="carForecast")
plt.show()

题 2-4 绘制一个 3D 图像并用优化方法找出最值

In [ ]:
# import numpy as np
# from scipy.optimize import minimize
# import matplotlib.pyplot as plt


def himmelblau(_x):
    # 定义 himmelblau 函数，并将数组/矩阵作为一个参数传入
    return (_x[0] ** 2 + _x[1] - 11) ** 2 + (_x[0] + _x[1] ** 2 - 7) ** 2


# 利用 scipy.optimize.minimize 求解极小值并输出大小和坐标
intMin = minimize(
    himmelblau, np.array([0.0, 0.0]), method="nelder-mead", options={"disp": True}
)
print("极小值为:", int(intMin.fun))
print("极小值坐标为:", intMin.x)

# 绘制 3D 图像
x = np.linspace(-6, 6, 256, endpoint=True)
y = np.linspace(-6, 6, 256, endpoint=True)
X, Y = np.meshgrid(x, y)
Z = himmelblau([X, Y])

ax = plt.axes(projection="3d")
ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap="rainbow", alpha=0.5)
plt.show()